In [1]:
import os
import numpy as np
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns
#sns.set(style="darkgrid")
from keras.models import load_model

Using TensorFlow backend.


In [2]:
TIMESTEPS = 1000
MIN_FAULT_DURATION = 12

In [3]:
df = pd.read_excel("time_series_all.xlsx")

In [4]:
df = df.dropna().reset_index(drop=True)

In [5]:
for i in df.columns[2:]:
    if(df[df.iloc[:,i]=='External identity error'].shape[0] != 0):
        df = df[df.iloc[:,i]!='External identity error']

c:\users\manik\appdata\local\programs\python\python36\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
# reduce resoolution
l = list(range(1,14000,4))
l.insert(0,'Feeder')
l.insert(1,'Measurand')

In [7]:
dff = df[l]

In [8]:
dff.set_index(['Feeder', 'Measurand'])

1         5      \
Feeder                                   Measurand                       
0                                        KV                0         0   
B-4 S.J. ENCL.                           AMPs         59.279   55.2146   
                                         PF        -0.981392 -0.981995   
IGNOU S/S -3(Feeder 1)                   AMPs        50.6842   47.0453   
                                         PF        -0.996355 -0.996485   
CKT 2 PAPPANKALAN                        AMPs        111.635   97.6701   
                                         KV          70.1961   70.2288   
                                         PF        -0.993118 -0.990419   
CKT 2 MEHRAUILI I/C                      AMPs         57.005   54.5396   
                                         PF        -0.988547 -0.986929   
BHAIRON RD                               AMPs        33.5153   31.6728   
                                         PF         0.981325  0.981325   
Incomer 2 MEHRAULI CKT2                  AMPs              0         0   
                                         KV          67.1221   67.6275   
                                         PF               -1        -1   
Incomer  OKHLA CKT 4                     AMPs              0         0   
                                         KV          34.1449   34.2107   
                                         PF               -1        -1   
S/S-11 D-4 V.KUNJ                        AMPs        53.0218   48.6972   
                                         PF        -0.969788 -0.974108   
DMRC + CARRIAPPA MARG SAINIK FORM        AMPs        9.15866   8.50337   
                                         PF        -0.992352 -0.996645   
SUBHAS VIHAR-1                           AMPs        69.4177   69.4177   
                                         PF         0.987461  0.987461   
CAPBANK-4                                AMPs              0         0   
                                         KV          10.8282    10.928   
CPWD SS5A                                AMPs              0         0   
                                         PF               -1        -1   
PUNJABI BASTI                            AMPs        48.6398   48.6398   
                                         PF        -0.997177 -0.997177   
...                                                      ...       ...   
B-1/283 VISH GAD                         AMPs         32.807   34.4186   
                                         PF        -0.959744 -0.895139   
1616JANTA FLATS                          AMPs             59        59   
                                         PF                0         0   
T-OFF MATIYALA PANKH.                    AMPs        23.5459    19.391   
                                         KV          64.9818   64.9028   
VENKATSWARA HOSPITAL-1                   AMPs              0         0   
                                         PF                0         0   
PLOT 8 DC                                AMPs        9.39418   7.71328   
                                         PF               -1        -1   
SEC-1 S/S-1 PUSHP VHR                    AMPs        66.6777   63.1926   
                                         PF         0.969092  0.968378   
J.J KHANPUR                              AMPs        101.691   95.5379   
                                         PF        -0.960588 -0.959253   
LODHI ROAD I/C NO.2                      AMPs       0.239037  0.201137   
                                         KV          33.8022   33.9751   
                                         PF               -1        -1   
400KV MUNDKA CKT                         AMPs        235.961   207.841   
                                         KV          11.2356   11.1912   
                                         PF         0.988105        -1   
KHANPUR KATTAWALA                        AMPs        77.9044   72.7136   
                                         PF        -0.97335

In [9]:
feeders = []
for i in dff['Feeder'].unique():
    tdf = dff[df['Feeder']==i]
    measurands = [tdf[tdf['Measurand']=='AMPs'].values, tdf[tdf['Measurand']=='KV'].values, tdf[tdf['Measurand']=='PF'].values]
    if len(measurands[0])==0: continue
    feeder = []
    for m in measurands:
        if len(m) != 0:
            feeder.append(m[0][2:])
        else:
            feeder.append(-np.ones(3500))
                
    feeders.append(feeder)
feeders = np.array(feeders)
feeders.shape

(1329, 3, 3500)

In [10]:
def isSubArray(A, n, m, st):
    B = [0] * m
    ii = st
    j = 0
    while (ii < n and j < m): 
        if (A[ii] == B[j]): 
            ii += 1 
            j += 1 
            if (j == m): 
                return ii,True
        else: 
            ii = ii - j + 1 
            j = 0 
    return ii,False 

In [11]:
l = []
for i in range(len(feeders)):
    xx = feeders[i][0][:TIMESTEPS]
    r = isSubArray(xx.tolist(), TIMESTEPS, MIN_FAULT_DURATION,0)
    if(r[1]):
        l.append((i, r[0]))                      # i=index, r[0]='timestamp', r[1]=fault
l = pd.DataFrame(l)
l

,0,1
0,2,596
1,3,433
2,5,85
3,6,12
4,7,229
5,10,12
6,11,12
7,13,626
8,14,582
9,18,672


In [12]:
indices = list(range(len(feeders)))
for i in reversed(l[0]):
    indices.pop(i)
feeders = feeders[indices]
len(feeders)

797

In [13]:
# faults
l = []
for i in range(len(feeders)):
    xx = feeders[i][0][TIMESTEPS:]
    r = isSubArray(xx.tolist(), len(xx), MIN_FAULT_DURATION,0)
    if(r[1]):
        l.append((i, r[0]))
l = pd.DataFrame(l)

# l: 0=index, 1='timestamp', 2=fault
X_fault_indices = pd.DataFrame(zip(l[0], l[1], l[1].apply(
    lambda t: list(range(t-MIN_FAULT_DURATION, t+TIMESTEPS-MIN_FAULT_DURATION))))) # l[1] is already fault - TIMESTEPS
X_fault = []
for i in X_fault_indices.values:
    X_fault.append(feeders[i[0]][:,i[2]])
    
X_fault = np.array(X_fault)                 # 0 - df index for future use
#X_fault = pd.DataFrame(list(X_fault[1].values))

X_fault.shape #.reshape(X_fault.shape[0], X_fault.shape[1], X_fault.shape[2], 1)

(292, 3, 1000)

In [14]:
indices = list(range(len(feeders)))
for i in reversed(l[0]):
    indices.pop(i)
no_faults = feeders[indices]
no_faults.shape

X_no_faults = []
for i in range(len(no_faults)):
    r = np.random.randint(feeders.shape[2] - TIMESTEPS)
    X_no_faults.append(no_faults[i][:, r:r+TIMESTEPS])
    
X_no_faults = np.array(X_no_faults)
X_no_faults.shape

(505, 3, 1000)

In [15]:
X = np.concatenate([X_no_faults, X_fault])
y = np.array([0]*len(X_no_faults) + [1]*len(X_fault))

X.shape, y.shape

((797, 3, 1000), (797,))

In [16]:
X = np.transpose(X, (0, 2, 1))
X.shape

(797, 1000, 3)

In [17]:
# Spliting into traing and testing 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
x_train.shape,x_test.shape

((637, 1000, 3), (160, 1000, 3))

In [ ]:
models = []
for i in range(50):
    dropout = np.random.random()/3
    model = Sequential()
    model.add(LSTM(np.random.randint(512), dropout = dropout, recurrent_dropout=dropout, 
                   input_shape=(x_train.shape[1],x_train.shape[2])))
    model.add(Dense(np.random.randint(16), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss = 'binary_crossentropy' , optimizer='adam',metrics=['accuracy'])
    history = model.fit(x_train,y_train,epochs=50,validation_split=0.2,batch_size=1000)
    models.add(model, history)
    print(history)

Train on 509 samples, validate on 128 samples
Epoch 1/50
